딥러닝 텍스트 분류 파이프라인
우리가 앞으로 진행할 텍스트 분류 딥러닝 모델 개발 과정은 크게 4단계로 나눌 수 있습니다.

1. 데이터 준비 및 전처리:

데이터셋을 불러오고, 필요한 열(column)을 확인합니다.

딥러닝 모델이 이해할 수 있도록 텍스트 데이터를 정제합니다. 불필요한 기호 제거, 소문자 변환, 토큰화(tokenization) 등이 이 단계에 해당합니다.

2. 단어 임베딩(Word Embedding):

정제된 텍스트 데이터의 각 단어를 의미를 담은 벡터로 변환합니다.

Word2Vec이나 GloVe와 같은 모델을 직접 학습하거나, 이미 학습된 모델을 불러와 활용할 수 있습니다.

3. 딥러닝 모델 구축:

PyTorch나 TensorFlow와 같은 딥러닝 프레임워크를 사용해 모델을 만듭니다.

임베딩 레이어(Embedding Layer), LSTM 레이어(LSTM Layer), 그리고 **완전 연결 레이어(Fully Connected Layer)**를 순서대로 쌓아 딥러닝 모델을 구성합니다.

4. 모델 학습 및 평가:

학습 데이터를 모델에 넣어 가중치(weight)를 최적화하고, 손실(loss)을 줄이는 과정을 반복합니다.

학습이 완료된 모델의 성능을 검증 데이터(validation data)와 테스트 데이터(test data)로 평가합니다.

1. 데이터 불러오기

설명: 훈련 및 테스트 데이터를 CSV 파일에서 pandas DataFrame으로 로드.
사용된 라이브러리 및 함수:

pandas:

pd.read_csv(): CSV 파일을 DataFrame으로 로드.




파일 경로:

훈련 데이터: train.csv
테스트 데이터: test.csv



2. 탐색적 데이터 분석 (EDA)

설명: 데이터의 구조와 특성을 파악하기 위해 보고서를 생성하고, keyword와 location 컬럼의 통계 정보를 확인.
사용된 라이브러리 및 함수:

(주석 처리됨) dataprep.eda.create_report(): 데이터 보고서 생성.
(주석 처리됨) report.save(): 보고서를 HTML 파일로 저장.


인사이트:

keyword: 221개 고유 값, 결측치 0.8% (61개).
location: 3341개 고유 값, 결측치 33.3% (2533개).



3. 텍스트 정제

3.1. keyword 컬럼 결측치 처리:

설명: keyword 컬럼의 결측치를 최빈값(mode())으로 대체.
사용된 함수:

pandas.DataFrame.fillna(): 결측치를 지정된 값으로 채움.
pandas.Series.mode(): 최빈값 계산.
pandas.DataFrame.isnull().sum(): 결측치 수 확인.




3.2. 대문자를 소문자로 변환:

설명: text 컬럼의 문자열을 소문자로 변환.
사용된 함수:

pandas.Series.str.lower(): 문자열을 소문자로 변환.




3.3. 특수문자, 숫자, 기호 제거:

설명: text 컬럼에서 알파벳과 공백을 제외한 모든 문자 제거.
사용된 함수:

pandas.Series.str.replace(): 정규 표현식을 사용해 패턴 대체.





4. 토큰화

설명: text 컬럼의 문자열을 단어 단위로 토큰화.
사용된 라이브러리 및 함수:

nltk:

nltk.tokenize.word_tokenize(): 문자열을 단어 단위로 분리.
nltk.download(): punkt, stopwords 데이터 다운로드.


pandas.DataFrame.apply(): 각 행에 함수 적용.


결과: tokenized_text 컬럼 생성.

5. 불용어 제거

설명: 토큰화된 텍스트에서 불용어(영어 stopwords) 제거.
사용된 라이브러리 및 함수:

nltk.corpus.stopwords: 영어 불용어 리스트 제공.
List comprehension: 불용어가 아닌 단어만 필터링.
pandas.DataFrame.apply(): 불용어 제거 함수 적용.


결과: non_stopwords_text 컬럼 생성.
후처리: 토큰 리스트를 문자열로 변환해 processed_text 컬럼 생성.

lambda 및 str.join(): 토큰 리스트를 공백으로 연결.



6. 벡터화 (Word2Vec)

설명: 불용어 제거된 텍스트를 Word2Vec으로 임베딩 벡터로 변환.
사용된 라이브러리 및 함수:

gensim.models.Word2Vec:

Word2Vec(): Word2Vec 모델 학습 (매개변수: vector_size=100, window=5, min_count=5, workers=4).
model.wv.most_similar(): 유사 단어 확인.
model.wv[]: 특정 단어의 임베딩 벡터 조회.




결과: 단어 사전 생성 및 단어 간 유사성 확인 (예: disaster 단어의 유사 단어).

7. 정수 인코딩 및 패딩

7.1. 정수 인코딩:

설명: 토큰화된 단어를 Word2Vec 단어 사전의 인덱스로 변환.
사용된 함수:

Dictionary comprehension: word_to_index 생성.
List comprehension: 토큰을 인덱스로 변환.
pandas.DataFrame.apply(): 정수 인코딩 적용.


결과: encoded_text 컬럼 생성.


7.2. 패딩:

설명: 모든 시퀀스의 길이를 가장 긴 시퀀스 길이(max_len)에 맞춰 0으로 패딩.
사용된 라이브러리 및 함수:

numpy.array(): 패딩된 시퀀스를 배열로 변환.
torch.LongTensor(): NumPy 배열을 PyTorch 텐서로 변환.
Custom function (pad_sequence): 시퀀스 패딩.


결과: X (입력 텐서), y (타겟 텐서) 생성.



8. 데이터 분할

설명: 데이터를 학습용(80%)과 검증용(20%)으로 분할.
사용된 라이브러리 및 함수:

sklearn.model_selection.train_test_split: 데이터 분할 (매개변수: test_size=0.2, random_state=42).


결과: X_train, X_val, y_train, y_val 생성.

9. 데이터 로더 구축

설명: PyTorch의 Dataset과 DataLoader를 사용해 데이터를 배치 단위로 처리.
사용된 라이브러리 및 함수:

torch.utils.data.Dataset:

Custom class TextDataset: 데이터와 레이블을 묶음.
__len__: 데이터셋 크기 반환.
__getitem__: 인덱스별 데이터/레이블 반환.


torch.utils.data.DataLoader:

DataLoader(): 배치 생성 (매개변수: batch_size=64, shuffle=True/False).



In [ ]:
import pandas as pd

# 1. 데이터 불러오기
# 데이터 불러오기 (DataFrame 객체로 로드)
train_path = '/Users/rick/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/NLP with Disaster Tweets/data/train.csv'
train_df = pd.read_csv(train_path) # train 데이터를 DataFrame으로 로드
test_path = '/Users/rick/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/NLP with Disaster Tweets/data/test.csv'
test_df = pd.read_csv(test_path)

# 2. EDA
# 보고서 생성 (target 인자 없이)
# from dataprep.eda import create_report # dataprep 데이터 분석
# report = create_report(train_df) # 여기서 target 인자를 삭제했습니다.
# 보고서를 HTML 파일로 저장
# 파일명은 train_dataprep_report.html로 지정합니다.
# report.save('train_dataprep_report.html')

# 데이터 분석
# 1.keyword 칼럼:
# Approximate Distinct Count: 221개
# Approximate Unique (%): 2.9%
# Missing: 61개
# Missing (%): 0.8%
# Memory Size: 556863
# 인사이트: sweetviz와 유사하게 221개의 고유 키워드가 있고, 약 0.8%의 적은 결측치가 존재합니다. 이 칼럼은 충분히 활용 가치가 있어 보입니다. 결측치는 처리해주는 것이 좋습니다.
# 2. location 칼럼:
# Approximate Distinct Count: 3341개
# Approximate Unique (%): 65.8%
# Missing: 2533개
# Missing (%): 33.3%
# Memory Size: 404598
# 인사이트: sweetviz에서 확인했던 것과 동일하게, 3341개의 매우 많은 고유 지역이 있으며, 무려 33.3%라는 높은 비율의 결측치가 존재합니다.

# 3. 텍스트 정제
# 3.1. keyword 컬럼 처리
train_df['keyword'] = train_df['keyword'].fillna(train_df['keyword'].mode()[0]) # fatalities
train_df.isnull().sum()
# 3.2. 대문자 > 소문자
train_df['text'] = train_df['text'].str.lower()
# 3.3. 특수문자, 숫자, 기호 제거
# r'[^a-z ]' 패턴을 사용하고, 대체할 문자열은 '' (빈 문자열)
train_df['text'] = train_df['text'].str.replace(r'[^a-z ]', '', regex=True)

# 4. 토큰화
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# NLTK 데이터 다운로드 확인
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('punkt')
    #nltk.download('punkt_tab')
    nltk.download('stopwords')
# 4.1 토큰화 (Tokenization)
# text 칼럼의 각 문자열에 word_tokenize 함수 적용
train_df['tokenized_text'] = train_df['text'].apply(word_tokenize)
# 토큰화 결과 확인
# print("토큰화 후 샘플:")
# for i in range(5):
#     print(f"Original Text (Cleaned): {train_df['text'][i]}")
#     print(f"Tokenized Text: {train_df['tokenized_text'][i]}\n")

# 5. 불용어 제거
stop_words = set(stopwords.words('english')) # 영어 중 불용어
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words] # 불용어가 아닌 단어
train_df['non_stopwords_text'] = train_df['tokenized_text'].apply(remove_stopwords)
# 불용어 제거 후 샘플
# for i in range(5):
#     print(f"Tokenized Text: {train_df['tokenized_text'][i]}")
#     print(f"Non-stopwords Text: {train_df['non_stopwords_text'][i]}\n")
# 토큰 리스트를 문자열로 변환
train_df['processed_text'] = train_df['non_stopwords_text'].apply(lambda tokens: ' '.join(tokens))

# 6. 벡터화 (Word2Vec 모델 사용)
from gensim.models import Word2Vec
# 토큰화 및 불용어 제거가 완료된 텍스트 데이터를 준비합니다.
# 이 데이터는 'non_stopwords_text' 열에 이미 들어있습니다.
sentences = train_df['non_stopwords_text'].tolist()
# Word2Vec 모델을 학습합니다.
# vector_size: 임베딩 벡터의 차원 (임베딩 공간의 크기)
# window: 주변 단어를 고려하는 윈도우 크기
# min_count: 최소 단어 빈도 (이 값보다 적게 등장하는 단어는 학습에서 제외)
# workers: 학습에 사용할 CPU 코어 수
embedding_model = Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=3,
    workers=4
)
# Word2Vec 모델의 가중치를 가져옵니다.
import torch
weights = torch.FloatTensor(embedding_model.wv.vectors)
# 'disaster' 단어의 임베딩 벡터 확인
# print("'disaster' 단어의 임베딩 벡터:", embedding_model.wv['disaster'])
# 'disaster'와 가장 유사한 단어 5개 찾기
similar_words = embedding_model.wv.most_similar('disaster', topn=5)
print("유사 단어:", similar_words)

# 7. 정수 인코딩 & 패딩
# 7.1. 정수 인코딩
# 정수 인코딩은 텍스트 데이터의 각 단어를 고유한 숫자로 변환하는 과정입니다. 이는 모델이 텍스트를 처리할 수 있도록 만들어 줍니다. 우리가 학습한 Word2Vec 모델에 이미 단어 사전이 포함되어 있어, 이 작업을 쉽게 할 수 있습니다.
# 7.1.1. Word2Vec 모델의 단어 사전을 가져옵니다.
word_to_index = {word: idx for idx, word in enumerate(embedding_model.wv.index_to_key)}
# 7.1.2. 토큰화된 텍스트 데이터를 정수 인코딩합니다.
def encode_text(tokens, word_to_index):
    return [word_to_index[token] for token in tokens if token in word_to_index]
train_df['encoded_text'] = train_df['non_stopwords_text'].apply(lambda x: encode_text(x, word_to_index))
# print("정수 인코딩 후 샘플:")
# print(train_df['encoded_text'].iloc[0])
# 7.2. 패딩
# 패딩을 통해 모든 트윗의 길이를 동일하게 맞춰줍니다. 딥러닝 모델이 데이터를 효율적으로 처리할 수 있도록 형태를 표준화하는 작업입니다. 가장 긴 문장의 길이를 기준으로 짧은 문장의 뒤에 0을 채워 넣습니다.
# 7.2.1. 가장 긴 문장 길이 찾기
max_len = max(len(s) for s in train_df['encoded_text'])
print(f"가장 긴 트윗의 길이: {max_len}")
# 7.2.2. 패딩을 위한 함수 정의
def pad_sequence(sequence, max_len):
    # 시퀀스 길이를 max_len으로 맞추고, 부족한 부분은 0으로 채웁니다.
    # PyTorch의 F.pad를 사용하거나, Python 리스트 조작으로 구현할 수 있습니다.
    # F.pad는 텐서 형태에서 사용. 아래는 Python 리스트로 구현
    padded = sequence + [0] * (max_len - len(sequence))
    return padded
# 7.2.3. 모든 문장에 패딩 적용
padded_sequences = [pad_sequence(seq, max_len) for seq in train_df['encoded_text']]
# 7.2.4. NumPy 배열로 변환 후 PyTorch 텐서로 변환
import numpy as np
padded_sequences = np.array(padded_sequences)
X = torch.LongTensor(padded_sequences)
# Y (타겟 변수)도 텐서로 변환
y = torch.LongTensor(train_df['target'].values)
# print("패딩 후 데이터 형태:", X.shape)
# print("패딩 후 샘플:", X[0])

# 8. 데이터 분할
# Word2Vec 기반 정수 인코딩 및 패딩된 시퀀스와 목표 변수 'target'을 학습용, 검증용으로 나눕니다.
# test_size는 검증용 데이터의 비율을 의미합니다. (0.2는 20%)
# random_state는 재현성을 위해 고정합니다.
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)
# 데이터 로더 구축 전에 텐서를 디바이스로 이동
# GPU 사용 설정 (맥북은 'mps')
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")
# 텐서를 디바이스로 이동시키는 코드 추가
X_train = X_train.to(device)
X_val = X_val.to(device)
y_train = y_train.to(device)
y_val = y_val.to(device)

# 9.데이터 로더 구축
# Dataset: 데이터셋의 **데이터(X)**와 **정답 레이블(Y)**을 묶어주는 역할을 합니다. 각 데이터 샘플에 쉽게 접근할 수 있도록 만들어 주죠.
# DataLoader: Dataset을 감싸서 미니 배치(mini-batch) 단위로 데이터를 자동으로 묶어주고, 학습 과정에서 데이터를 모델에 전달하는 역할을 합니다. 이렇게 하면 전체 데이터를 한꺼번에 메모리에 올리지 않고도 효율적으로 학습할 수 있어요.
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        # 데이터셋의 전체 길이를 반환
        return len(self.X)
    def __getitem__(self, idx):
        # 주어진 인덱스(idx)에 해당하는 데이터와 레이블을 반환
        return self.X[idx], self.y[idx]
# 데이터셋 객체 생성
train_dataset = TextDataset(X_train, y_train)
val_dataset = TextDataset(X_val, y_val)
# DataLoader 객체 생성
BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
print(f"학습 데이터로더 배치 수: {len(train_loader)}")
print(f"검증 데이터로더 배치 수: {len(val_loader)}")
# 첫 번째 배치 확인
for X_batch, y_batch in train_loader:
    print(f"첫 번째 배치 데이터 형태: {X_batch.shape}")
    print(f"첫 번째 배치 레이블 형태: {y_batch.shape}")
    break

# 10. 딥러닝 모델 구축
import torch.nn as nn # PyTorch의 신경망 모듈(패키지)
class LSTMClassifier(nn.Module): # nn.Module을 상속받아 LSTMClassifier 클래스 정의
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, pretrained_weights=None):
        # vocab_size: 단어 사전의 크기. 모델이 알아야 할 단어의 총 개수
        # embedding_dim: 각 단어를 표현하는 임베딩 벡터의 차원, 각 단어를 몇 차원의 벡터로 표현할지를 결정.
        # hidden_dim: LSTM의 은닉 상태(hidden state) 차원. LSTM 층이 학습하는 정보의 '깊이'를 결정.
        # output_dim: 모델의 출력 차원. 분류 문제에서는 클래스의 수와 동일. 지금은 이진 분류이므로 0(재난 트윗 아님) 또는 1(재난 트윗)을 나타낼 수 있도록 보통 1로 설정.
        # num_layers: LSTM 층을 설정. 기본 1개.
        super().__init__() # 부모 클래스 초기화
        # 레이어(층) 정의
        # 임베딩 레이어
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0) # 패딩 인덱스 0으로 설정. 패딩으로 설정된 0을 학습에서 제외시키기 위해서 설정.
        # Word2Vec 가중치가 있으면 로드
        if pretrained_weights is not None:
            self.embedding.weight.data.copy_(pretrained_weights)
            # Fine-tuning 허용 (학습 중 임베딩도 업데이트 됨)
            self.embedding.weight.requires_grad = True
        # LSTM 레이어
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True) # num_layers 추가: 층수만큼 스택
        # 완전연결 레이어
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, text):
        # text = [batch size, seq len]
        embedded = self.embedding(text)
        # 입력 데이터 text를 임베딩 층에 통과시켜 단어 벡터로 변환합니다.  이 결과로 [batch size, seq len, embedding dim] 형태의 3차원 텐서가 생성되는데, 이것은 "여러 문장(batch size)에 있는 각 단어(seq len)를 벡터(embedding dim)로 표현했다"는 뜻입니다.
        # embedded = [batch size, seq len, embedding dim]
        output, (hidden, cell) = self.lstm(embedded)
        # 임베딩된 텐서가 LSTM 층을 통과합니다. 이때 LSTM은 두 가지를 출력합니다.
        # output: 각 시점(time step)의 출력.
        #hidden: 마지막 시점의 은닉 상태(Hidden State).
        # cell: 마지막 시점의 셀 상태(Cell State).
        # output = [batch size, seq len, hidden dim]
        # hidden = [1, batch size, hidden dim]
        # hidden = hidden.squeeze(0)
        # hidden 텐서의 형태가 [1, batch size, hidden dim]인데, LSTM은 기본적으로 하나의 층을 사용하므로 맨 앞의 차원(1)은 필요가 없습니다. squeeze(0)를 사용해 불필요한 차원을 제거하여 [batch size, hidden dim] 형태로 만들어줍니다.
        # hidden = [batch size, hidden dim]
        hidden = hidden[-1]  # [num_layers, batch_size, hidden_dim]의 마지막 층 추출: [batch_size, hidden_dim]
        return self.fc(hidden)
        # 마지막으로, 이렇게 정리된 hidden 텐서를 완전 연결 층(fc)에 넣어 최종 분류 결과를 얻습니다. 이때 hidden은 "문장 전체의 의미를 압축한 벡터"라고 생각할 수 있습니다.
import torch.optim as optim
# 하이퍼파라미터 정의
vocab_size = len(word_to_index) # 단어 사전 크기
embedding_dim = 100 # vector_size
hidden_dim = 512 # LSTM의 은닉 상태 크기(모델이 정보를 기억하여 복잡한 패턴을 학습)
num_layers = 2 # LSTM 층 개수를 설정. 기본 1, 증가 시 모델 깊이 증가로 복잡한 시퀀스 패턴 학습 강화)
output_dim = 1 # 최종 출력 차원, 이진 분류이므로 1로 설정
# Word2Vec 가중치를 PyTorch 텐서로 변환
pretrained_weights = torch.FloatTensor(embedding_model.wv.vectors)
# 모델 객체 생성
model = LSTMClassifier(vocab_size
                     , embedding_dim
                     , hidden_dim
                     , output_dim
                     , num_layers
                     , pretrained_weights=pretrained_weights # 가중치 적용
                       )
# 손실 함수 정의: BCEWithLogitsLoss(이진 교차 엔트로피 손실 함수)
criterion = nn.BCEWithLogitsLoss()
# 최적화 정의: Adam(최적화 알고리즘)
optimizer = optim.Adam(model.parameters())

# 11. 모델 학습 및 평가 (훈련 루프)
# 학습 루프(Training Loop)
# 모델이 데이터로부터 지식을 습득하는 과정입니다. 데이터를 미니 배치 단위로 모델에 주입하고, 모델의 예측과 실제 정답 사이의 차이(손실)를 계산합니다. 이 손실을 줄이기 위해 역전파(Backpropagation)를 통해 모델의 가중치들을 조금씩 조정합니다.
# 평가 루프(Validation Loop)
# 모델이 학습 데이터에만 과적합되지 않았는지 확인하는 과정입니다. 학습이 끝날 때마다 별도로 분리해 둔 검증 데이터셋으로 모델의 성능을 평가합니다. 이때는 가중치를 업데이트하지 않고 오직 예측 정확도만 계산합니다.
def train_model(model, train_loader, optimizer, criterion):
    # 모델 훈련의 핵심 부분으로, 미니 배치 단위로 학습.
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for inputs, labels in train_loader:
        # 1. 그래디언트 초기화: 이전 배치의 그래디언트를 초기화
        # 그래디언트는 기울기또는 변화율을 말한다.
        optimizer.zero_grad()
        # 2. 예측값 계산: 입력 데이터를 모델에 넣어 계산
        outputs = model(inputs)
        # 3. 손실 계산: 예측값과 정답을 비교하여 손실을 계산
        loss = criterion(outputs.squeeze(1), labels.float()) # 텐서의 형태를 위해 형태 변환
        # 4. 역전파: 계산된 손실을 사용하여 모델의 모든 학습 가능한 매개변수에 대한 그래디언트를 계산. 손실을 최소화 하는 방향 제시. 역전파가 그래디언트를 계산하는 알고리즘이고 계산된 그래디언트를 사용하여 가중치를 업데이트하는 최적화 알고리즘을 경사하강법이라고 한다.
        loss.backward()
        # 5. 가중치 업데이트: 옵티마이저는 역전파를 통해 계산된 그래디언트 값을 사용하여 모델의 모든 가중치를 업데이트.
        optimizer.step()
        # 예를 들어, 3번 채점하여 4번에서 어디를 어떻게 공부할지 분석 5번 실제로 공부하여 실력 향상
        # 훈련 진행 상황 추적(1~5까지 한 에포크)
        total_loss += loss.item()
        preds = torch.round(torch.sigmoid(outputs)) # FC 층의 출력은 제한이 없다. 음수, 양수 등 아무거나 나올 수 있는데 우리는 0 or 1을 원하기 때문에 sigmoid 함수를 적용(sigmoid는 확률로 해석이 가능)시켜 0~1 사이 값으로 표현되게 변경하여 반올림해서 0 또는 1로 만들어 준다.
        total_correct += (preds.squeeze(1) == labels).sum().item()
        total_samples += labels.size(0)
    avg_loss = total_loss / len(train_loader)
    accuracy = total_correct / total_samples
    return avg_loss, accuracy

def evaluate_model(model, val_loader, criterion):
    # 평가 함수
    model.eval() # 모델을 평가모드로 전환. Dropout이나 BatchNorm 같은 일부 층들이 비활성화
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():
        # 학습이 아닌 결과 값을 확인해야하니 그래디언트 계산을 비활성화하여 가중치가 변경되지 않도록 함.
        for inputs, labels in val_loader:
            # 1. 예측값 계산
            outputs = model(inputs)
            # 2. 손실 계산
            loss = criterion(outputs.squeeze(1), labels.float())
            # 훈련 진행 상황 추적
            total_loss += loss.item()
            preds = torch.round(torch.sigmoid(outputs))
            total_correct += (preds.squeeze(1) == labels).sum().item()
            total_samples += labels.size(0)
    avg_loss = total_loss / len(val_loader)
    accuracy = total_correct / total_samples
    return avg_loss, accuracy

# 에포크 수 설정: 모델을 5회 반복(10회 반복 시, 과적합 진행)
N_EPOCHS = 5
# 학습 및 평가 결과 저장을 위한 리스트
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
# GPU 사용 설정 (맥이기에 'mps')
# device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

# 전체 학습 루프: 핵심 반복문
for epoch in range(N_EPOCHS):
    # 훈련
    train_loss, train_acc = train_model(model, train_loader, optimizer, criterion)
    # 검증
    val_loss, val_acc = evaluate_model(model, val_loader, criterion)
    # 결과 저장
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    # 결과 출력
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tVal. Loss: {val_loss:.3f} | Val. Acc: {val_acc*100:.2f}%')


# 1. 테스트 데이터 불러오기
test_path = '/Users/rick/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/NLP with Disaster Tweets/data/test.csv'
test_df = pd.read_csv(test_path)
print("테스트 데이터 로드 완료.")

# 2. 테스트 데이터 전처리 (훈련 데이터와 동일한 방식 적용)
# 2.1. keyword 결측치 처리: **훈련 데이터(train_df)의 최빈값**으로 채웁니다.
keyword_mode = train_df['keyword'].mode()[0]
test_df['keyword'] = test_df['keyword'].fillna(keyword_mode)

# 2.2. text 컬럼 소문자 변환 및 특수문자 제거
test_df['text'] = test_df['text'].str.lower()
test_df['text'] = test_df['text'].str.replace(r'[^a-z ]', '', regex=True)

# 2.3. 토큰화 및 불용어 제거
test_df['tokenized_text'] = test_df['text'].apply(word_tokenize)
test_df['non_stopwords_text'] = test_df['tokenized_text'].apply(remove_stopwords)
print("텍스트 정제, 토큰화, 불용어 제거 완료.")

# 3. 정수 인코딩 및 패딩
# 3.1. 정수 인코딩: **훈련 시 생성된 단어 사전(word_to_index)**을 사용합니다.
test_df['encoded_text'] = test_df['non_stopwords_text'].apply(lambda x: encode_text(x, word_to_index))

# 3.2. 패딩: **훈련 데이터 기준 최대 길이(max_len)**로 길이를 맞춥니다.
test_padded_sequences = [pad_sequence(seq, max_len) for seq in test_df['encoded_text']]

# 3.3. 최종 텐서 변환
X_test = torch.LongTensor(np.array(test_padded_sequences))
X_test = X_test.to(device)
print("정수 인코딩, 패딩, 텐서 변환 완료.")
print(f"최종 테스트 데이터 형태: {X_test.shape}")

# 4. 테스트 데이터 로더 생성
# 테스트용 Dataset은 레이블(y)이 필요 없습니다.
class TestDataset(Dataset):
    def __init__(self, X):
        self.X = X
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx]

test_dataset = TestDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
print("테스트 데이터 로더 생성 완료.")

# 5. 모델 예측 수행
# 5.1. 모델을 평가 모드로 전환 (매우 중요!)
model.eval()

# 5.2. 예측 결과를 담을 리스트 생성
test_preds = []

# 5.3. 그래디언트 계산 비활성화
with torch.no_grad():
    for inputs in test_loader:
        # 모델 예측
        outputs = model(inputs)
        
        # 예측값을 0 또는 1로 변환 (sigmoid -> 0.5 기준으로 반올림)
        preds = torch.round(torch.sigmoid(outputs))
        
        # 결과를 리스트에 추가 (CPU로 이동 후 NumPy 배열로 변환)
        test_preds.extend(preds.cpu().numpy().flatten().astype(int))

print("모델 예측 완료.")
print(f"총 예측 개수: {len(test_preds)}")


# 6. 제출 파일(submission.csv) 생성
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'target': test_preds
})

submission_path = 'submission.csv'
submission_df.to_csv(submission_path, index=False)

print(f"✅ '{submission_path}' 파일이 성공적으로 생성되었습니다.")
print("제출 파일 샘플:")
print(submission_df.head())

유사 단어: [('im', 0.9990947246551514), ('still', 0.9990237355232239), ('like', 0.9990084171295166), ('us', 0.9990033507347107), ('fire', 0.9989922046661377)]
가장 긴 트윗의 길이: 20
Using device: mps
학습 데이터로더 배치 수: 96
검증 데이터로더 배치 수: 24
첫 번째 배치 데이터 형태: torch.Size([64, 20])
첫 번째 배치 레이블 형태: torch.Size([64])


AttributeError: 'Word2Vec' object has no attribute 'mv'